In [1]:
import pandas as pd
from pathlib import Path
import sys
sys.path.append("../src")

import pull_solar_count_and_capacity as eia
from solar_technical_potential import compute_solar_eligibility
from median_solar_costs import load_and_process_lbnl
from average_electricity_prices import build_eia861_prices
from median_permit_fees import run_pipeline as fees_runpipeline
from median_interconnection_timelines import run_pipeline as ix_runpipeline
from solar_bill_savings import compute_state_bill_savings

In [2]:
# Load data

# ResStock metadata for solar potential calcs
resstock_metadata = pd.read_csv("../data/resstock_metadata.csv")

# AHJ population data
ahj_pop = pd.read_csv("../data/ahj_distribution_permitting_timelines_pv.csv")[['state', 'name', 'geoid', 'population']]

# Solar TRACE permitting fees
fees = pd.read_csv("../data/solartrace_fees.csv")

# Solar TRACE interconnection timelines
ix = pd.read_csv("../data/solartrace_ix.csv")

# State name <> abbr mapping
name_abbr = pd.read_csv("../data/state_name_abbr.csv")


/var/folders/sr/hy8nzjhj7xz25lt74m44nv140000gn/T/ipykernel_11091/993735047.py:4: DtypeWarning: Columns (102,103,105,144) have mixed types. Specify dtype option on import or set low_memory=False.
  resstock_metadata = pd.read_csv("../data/resstock_metadata.csv")


In [3]:
# Solar/Storage capacity and installations by year by sector

years = range(2017, 2026)

df = eia.download_and_aggregate_distributed_solar(years)

In [4]:
# Solar eligibility

state_results = compute_solar_eligibility(resstock_metadata)


In [ ]:
# Median solar costs over time

# Google Drive link to the LBNL Tracking the Sun ZIP file
drive_link = "https://drive.google.com/file/d/1NQh4TRC_IqDz2r5vfZuxDm6LGjEuexdu/view"

# FRED API Key
fred_api_key = "2764715428a4687d2a8ce57af948081d"

results = load_and_process_lbnl(
    drive_url=drive_link,
    fred_api_key=fred_api_key,
    start_year=2000,        # CPI start year
    min_install_year=2000,  # Only installations from 2000 onward
)


Downloading...
From (original): https://drive.google.com/uc?id=1NQh4TRC_IqDz2r5vfZuxDm6LGjEuexdu
From (redirected): https://drive.google.com/uc?id=1NQh4TRC_IqDz2r5vfZuxDm6LGjEuexdu&confirm=t&uuid=3dc1ef51-12e2-4f48-b25f-23e2e6a2eb84
To: /var/folders/sr/hy8nzjhj7xz25lt74m44nv140000gn/T/lbnl_1uxn8lxg/lbnl_latest.zip
100%|██████████| 218M/218M [00:08<00:00, 27.1MB/s] 


Extracting LBNL ZIP …
/var/folders/sr/hy8nzjhj7xz25lt74m44nv140000gn/T/lbnl_1uxn8lxg/TTS_LBNL_public_file_29-Sep-2025_all.csv
Loading LBNL CSV: /var/folders/sr/hy8nzjhj7xz25lt74m44nv140000gn/T/lbnl_1uxn8lxg/TTS_LBNL_public_file_29-Sep-2025_all.csv


/Users/talorgruenwald/src/permit_power/basic-solar-statistics/notebooks/../src/median_solar_costs.py:255: DtypeWarning: Columns (1,2,3,11,15,16,18,28,29,31,32,34,35,38,39,40,53,54,56,57,59,60,74,75,79,80) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(csv_path)


In [6]:
# Average electricity prices

years = range(2015, 2025)

util_prices, state_prices = build_eia861_prices(years)

/Users/talorgruenwald/src/permit_power/basic-solar-statistics/notebooks/../src/average_electricity_prices.py:419: FutureWarning: The default fill_method='ffill' in SeriesGroupBy.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  agg.groupby(["utility_number", "sector"])["price_per_kwh"].pct_change()


In [3]:
# Median permitting fees

ahj_df_fees, state_df_fees = fees_runpipeline(fees, ahj_pop, name_abbr)

In [4]:
# Median IX timelines

ahj_df_ix, state_df_ix = ix_runpipeline(ix)

In [2]:
# Average and lifetime bill savings

base_dir = "/Volumes/Seagate Portabl/permit_power/dgen_runs/per_state_outputs"

savings = compute_state_bill_savings(base_dir, run_name="run_all_states_net_savings_adjust_loan_params")